# Système non-linéaires

## Méthode de Newton

### Optimisation

Dans le cadre de l'optimisation, la mise à jour d'un itéré se fait comme suit:
$$
x_{k+1} = x_k-\nabla^2 f(x_k)^{-1} \nabla f(x_k).
$$
La récurrence de Newton, si elle convergence, donne un point $x^*$ tel que $\nabla f(x^*) = 0$.

### Systèmes non-linéaires

La méthode de Newton peut aussi être appliquée à des systèmes nonlinéaires plus généraux. Considérons
$$
F(x) = \begin{pmatrix} f_1(x) \\ f_2(x) \\ \vdots \\ f_m(x) \end{pmatrix}.
$$
Dans ce cas, nous pouvons appliquer la méthode de Newton pour cherche un point $x^*$ tel que $F(x^*) = 0$. Le récurrence ed Newton est alors
$$
x_{k+1} = x_k-J(x_k)^{-1} F(x_k).
$$
Il est facile d'adapter le code vu dans un contexte d'optimisation.

In [1]:
using LinearAlgebra

In [2]:
function NLSNewton(F:: Function, h:: Function,
        xstart::Vector, δ::Float64 = 1e-4, nmax::Int64 = 1000)

    k = 1
    x = xstart
    n = length(x)
    δ2 = δ*δ
    J = zeros(n,n)
    
    Fx = ones(n)
    Fx = F(x, Fx)

    while (dot(Fx,Fx) > δ2 && k <= nmax)
        k += 1
        Fx = F(x, Fx)
        J = h(x,J)
        # Js = Fx, x_{k+1} = x_k - s
        x -= J\Fx  # x = x - s
    end
    
    return x
end

NLSNewton (generic function with 3 methods)

Considérons par exemple le système non-linéaire
\begin{align*}
    4x+2y+2xz-10 &= 0 \\
    2x+y+yz-6 &= 0 \\
    x^2 + y^2 - 5 &= 0
\end{align*}
Nous implémentons d'abord la multi-fonction correspondante.

In [3]:
function F(x:: Vector, d:: Vector)
    
    if length(d) == 0
        d = zeros(length(x))
    end
    
    d[1] = 4*x[1]+2*x[2]+2*x[1]*x[3]-10
    d[2] = 2*x[1]+x[2]+x[2]*x[3]-6
    d[3] = x[1]*x[1]+x[2]*x[2]-5
    
    return d
end

F (generic function with 1 method)

Nous pouvons facilement vérifier que (1,2,1) satisfait ce système.

In [4]:
d = zeros(3)
x = [ 1; 2; 1]
h = F(x, d)
d

3-element Array{Float64,1}:
 0.0
 0.0
 0.0

La matrice jacobienne est
$$
\begin{pmatrix}
    4+2z & 2 & 2x \\
    2 & 1+z & y \\
    2x & 2y & 0
\end{pmatrix}
$$

In [5]:
function J(x::Vector, H::Matrix)

    (m,n) = size(H)
 
    if (m,n) == (0,0)
        n = length(x)
        H = zeros(n,n)
    end

    H[1,1] = 4+2*x[3]
    H[2,2] = 1+x[3]
    H[3,3] = 0.0
    H[1,2] = H[2,1] = 2
    H[1,3] = H[3,1] = 2*x[1]
    H[2,3] = x[2]
    H[3,2] = 2*x[2]
    
    return H
end

J (generic function with 1 method)

In [6]:
NLSNewton(F, J, x)

3-element Array{Int64,1}:
 1
 2
 1

In [7]:
x0 = [1,1,1.0]

3-element Array{Float64,1}:
 1.0
 1.0
 1.0

In [8]:
NLSNewton(F, J, x0)

3-element Array{Float64,1}:
 0.9999999999999999
 2.0
 1.0000000000000002

La méthode de Newton pour résoudre un système non-linéaire souffre cependant des mêmes problèmes que dans le contexte d'optimisation. Sa convergence n'est assurée que si le point de départ est dans un voisinage de la solution. La récurrence peut même être ne pas être bien définie.

In [10]:
NLSNewton(F, J, [0, 0 ,0.0])

LoadError: SingularException(2)

## Exemple 2

Nous cherchons à résoudre le problème
\begin{align*}
x^3 + y &= 1 \\
y^3 − x &= −1.
\end{align*}

La multi-fonction du sytème peut être définie par

In [11]:
function g2(x::Vector, d::Vector)

    if length(d) == 0
        d = zeros(length(x))
    end
    
    d[1] = x[1]^3+x[2]-1
    d[2] = x[2]^3-x[1]+1
    
    return d
end

g2 (generic function with 1 method)

In [12]:
d = [1.0, 1.0]
x = [1.0, 0]

2-element Array{Float64,1}:
 1.0
 0.0

In [13]:
g2(x, d)
d

2-element Array{Float64,1}:
 0.0
 0.0

In [14]:
function J2(x::Vector, H::Matrix)
    
    (m,n) = size(H)
 
    if (m,n) == (0,0)
        n = length(x)
        H = zeros(n,n)
    end

    H[1,1] = 3*x[1]^2
    H[1,2] = 1
    H[2,1] = -1
    H[2,2] = 3*x[2]^2

    return H
end

J2 (generic function with 1 method)

In [15]:
H = [0 0; 0 0]
J2(x,H)

2×2 Array{Int64,2}:
  3  1
 -1  0

In [16]:
x = [1.0, 1.0]
NLSNewton(g2, J2, x)

2-element Array{Float64,1}:
 0.9999999999999971
 1.8263043495851585e-12

Réalisaons la première itération à la main. Nous devons résoudre le système
$$
\begin{pmatrix}
3*1^2 & 1 \\ -1 & 3*1^2
\end{pmatrix}
\begin{pmatrix}
d_1 \\ d_2
\end{pmatrix}
=
\begin{pmatrix}
1^3+1-1 \\ 1^3-1+1
\end{pmatrix}
$$
ou
$$
\begin{pmatrix}
3 & 1 \\ -1 & 3
\end{pmatrix}
\begin{pmatrix}
d_1 \\ d_2
\end{pmatrix}
=
\begin{pmatrix}
1 \\ 1
\end{pmatrix}
$$
De manière équivalente, nous devons résoudre
\begin{align*}
3d_1 + d_2 &= 1 \\
-d_1 + 3d_2 &= 1 \\
\end{align*}
Nous en déduisons que
$$
10 d_2 = 4
$$
ou
$$
d_2 = 0.4
$$
et dès lors
$$
d_1 = 0.2
$$
Nous pouvons vérifier ce résultat en résolvant numériquement le système linéaire correspondant

In [17]:
A = [3 1 ; -1 3]
b = [1 ; 1]
A\b

2-element Array{Float64,1}:
 0.20000000000000004
 0.39999999999999997

Dès lors, l'itération de Newton donne
$$
\begin{pmatrix}
x_{k+1} \\ y_{k+1}
\end{pmatrix}
=
\begin{pmatrix}
1 \\ 1
\end{pmatrix}
-
\begin{pmatrix}
0.2 \\ 0.4
\end{pmatrix}
=
\begin{pmatrix}
0.8 \\ 0.6
\end{pmatrix}
$$

## Exemple 3

\begin{align*}
a+b+c &= 6 \\
a^2+b^2+c^2 &= 14 \\
a^3+b^3+c^3 &= 36
\end{align*}

In [18]:
function g3(x::Vector, d::Vector)

    if length(d) == 0
        d = zeros(length(x))
    end
    
    for i = 1:3
        d[i] = x[1]^i+x[2]^i+x[3]^i
    end
    d[1] -= 6
    d[2] -= 14
    d[3] -= 36

    return d
end

g3 (generic function with 1 method)

In [19]:
function J3(x::Vector, H::Matrix)

    (m,n) = size(H)
 
    if (m,n) == (0,0)
        n = length(x)
        H = zeros(n,n)
    end

    for i = 1:3
        for j = 1:3
            H[i,j] = i*x[j]^(i-1)
        end
    end
    return H
end

J3 (generic function with 1 method)

In [20]:
x = [1.0, 1.0, 1.0]
NLSNewton(g3, J3, x)

LoadError: SingularException(2)

In [21]:
H = ones(3,3)
J3(x,H)

3×3 Array{Float64,2}:
 1.0  1.0  1.0
 2.0  2.0  2.0
 3.0  3.0  3.0

In [22]:
rank(H)

1

In [23]:
x = [2.0, 1.0, 0.0]
NLSNewton(g3, J3, x)

3-element Array{Float64,1}:
 2.9999999999999996
 0.9999999999999994
 2.0000000000000013

## Application au conditions KKT

Considérons le problème contraint linéairement
\begin{align*}
\min\ & f(x) \\
\mbox{t.q. } & Ax = b
\end{align*}
où $A \in \mathbb{R}^{m \times n}$.

La fonction lagrangienne de ce problème est
$$
L(x,\mu) = f(x) + \sum_{i = 1}^m \mu_i(a_i^Tx-b_i)
$$
où $a_i$ désigne la $i^e$ ligne de la matrice et $b_i$ le $i^e$ élément de $b$.

Les conditions KKT de ce problème sont
\begin{align*}
\nabla f(x) + A^T\mu &= 0 \\
Ax - b &= 0
\end{align*}
Il s'agit d'un système non-linéaire qui peut être résolu en utilisant la méthode de Newton.

Considérons par exemple le problème
$$
\min f(x,y) = -10x^2+10y^2+4\sin(xy)-2x+x^4
$$

In [24]:
f = x -> -10*x[1]^2+10*x[2]^2+4*sin(x[1]*x[2])-2*x[1]+x[1]^4

function ∇f(x:: Vector, g:: Vector)
    g[1] = -20*x[1]+4*x[2]*cos(x[1]*x[2])-2+4*x[1]^3
    g[2] = 20*x[2]+4*x[1]*cos(x[1]*x[2])
    return g
end

function Hess(x:: Vector, H:: Matrix)
    H[1,1] = -20-4*x[2]^2*sin(x[1]*x[2])+12*x[1]^2
    H[2,1] = H[1,2] = 4*cos(x[1]*x[2])-4*x[1]*x[2]*sin(x[1]*x[2])
    H[2,2] = 20-4*x[1]^2*sin(x[1]*x[2])
    return H
end

Hess (generic function with 1 method)

In [25]:
using ForwardDiff

gr = x -> ForwardDiff.gradient(f, x);
He = x -> ForwardDiff.hessian(f, x)

function gr!(x::Vector, storage::Vector)
    s = gr(x)
    storage[1:length(s)] = s[1:length(s)]
end

function He!(x::Vector, storage::Matrix)
    s = He(x)
    n, m = size(s)
    storage[1:n,1:m] = s[1:n,1:m]
end

He! (generic function with 1 method)

In [28]:
x = zeros(2)

2-element Array{Float64,1}:
 0.0
 0.0

Nous pouvons résoudre le système KKT avec notre méthode de Newton, en utilisant les dérivées analytiques ou la différentiation automatique.

In [29]:
sol = NLSNewton(∇f, Hess, x)

2-element Array{Float64,1}:
 -0.09632573825923432
  0.019265114479965602

In [30]:
sol = NLSNewton(gr!, He!, x)

2-element Array{Float64,1}:
 -0.09632573825923432
  0.019265114479965602

In [33]:
grad = zeros(2)
hess = zeros(2,2)

2×2 Array{Float64,2}:
 0.0  0.0
 0.0  0.0

In [34]:
∇f(sol, grad)

2-element Array{Float64,1}:
 1.5612511283791264e-17
 0.0

In [35]:
Hess(sol, hess)

2×2 Array{Float64,2}:
 -19.8887    3.99998
   3.99998  20.0001

Calculons les valeurs propres de la matrice hessienne.

In [36]:
eigen(hess)

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
values:
2-element Array{Float64,1}:
 -20.28581085718777
  20.397226260858613
vectors:
2×2 Array{Float64,2}:
 -0.995107  0.098804
  0.098804  0.995107

Nous avons identifié un point selle!

Commençons avec un autre point de départ.

In [37]:
sol = [-2.21022, 0.329748]
∇f(sol, grad)

2-element Array{Float64,1}:
 -1.91503605222465e-5
 -1.5587288675789068e-5

In [39]:
x = [-10; -10.0]
sol = NLSNewton(gr!, He!, x)

2-element Array{Float64,1}:
 -2.210219520077777
  0.3297484569954491

In [40]:
sol = NLSNewton(∇f, Hess, x)

2-element Array{Float64,1}:
  2.3066301277034658
 -0.33230864873179117

In [41]:
sol = [2.30663, -0.332309]
∇f(sol, grad)

2-element Array{Float64,1}:
 -5.903555937436522e-6
 -1.2307122703170137e-5

In [42]:
x = [2.5, -0.3]
sol = NLSNewton(gr!, He!, x)

2-element Array{Float64,1}:
  2.306630127703466
 -0.3323086487317913

In [43]:
∇f(sol, grad)

2-element Array{Float64,1}:
  2.842170943040401e-14
 -5.329070518200751e-15

In [44]:
Hess(sol, hess)

2×2 Array{Float64,2}:
 44.1529     0.754635
  0.754635  34.7619

Considérons maintenant le programme contraint
\begin{align*}
\min\ & f(x,y) = -10x^2+10y^2+4\sin(xy)-2x+x^4 \\
\text{s.t. } & 0.1x+y=1
\end{align*}
La fonction lagrangienne est maintenant
$$
L(x,y,\mu) = -10x^2+10y^2+4\sin(xy)-2x+x^4 + \mu(0.1x+y-1)
$$
et les conditions KKT sont
\begin{align*}
-20x+4y\cos(xy)-2+4x^3 + 0.1\mu &= 0 \\
20y+4x\cos(xy) + \mu &= 0 \\
0.1x+y-1 &= 0
\end{align*}

Afin de résoudre ce système, développons la matrice jacobienne
$$
J(x,y,\mu) =
\begin{pmatrix}
-20 - 4y^2\sin(xy)+12x^2 & 4\cos(xy)-4xy\sin(xy) & 0.1 \\
4\cos(xy)-4xy\sin(xy) & 20 - 4x^2\sin(xy) & 1 \\
0.1 & 1 & 0
\end{pmatrix}
$$

Implémentons ces opérateurs.

In [45]:
function KKT!(x, K)
    K[1] = -20*x[1]+4*x[2]*cos(x[1]*x[2])-2+4*x[1]^3+0.1*x[3]
    K[2] = 20*x[2]+4*x[1]*cos(x[1]*x[2])+x[3]
    K[3] = 0.1*x[1]+x[2]-1
    return K
end

function J!(x, J)
    J[1,1] = -20-4*x[2]^2*sin(x[1]*x[2])+12*x[1]^2
    J[2,1] = J[1,2] = 4*cos(x[1]*x[2])-4*x[1]*x[2]*sin(x[1]*x[2])
    J[2,2] = 20-4*x[1]^2*sin(x[1]*x[2])
    J[3,3] = 0
    J[1,3] = J[3,1] = 0.1
    J[2,3] = J[3,2] = 1
    return J
end

J! (generic function with 1 method)

In [46]:
x = [2.5, -0.3, 1.0]
sol = NLSNewton(KKT!, J!, x)

3-element Array{Float64,1}:
   2.3298897130224083
   0.7670110286977592
 -13.340493481573121

In [47]:
x = [-2.5, -2.5, 1.0]
sol = NLSNewton(KKT!, J!, x)

3-element Array{Float64,1}:
  -1.9843163000402988
   1.19843163000403
 -29.702536010170014

In [49]:
k = ones(3)
k = KKT!(sol, k)
println("Valeur KKT: ", k)

Valeur KKT: [-1.6362600163688512e-10, -9.137579581874888e-12, 2.220446049250313e-16]


In [50]:
grad = ones(3)
∇f(sol, grad)

3-element Array{Float64,1}:
  2.9702536008533755
 29.702536010160877
  1.0

Nous avons trouvé deux solutions au problème KKT.

Notons que dans cet exemple, nous pouvons aussi utiliser la contrainte linéaire pour éliminer une variable dans la fonction objectif, et avoir seulement à résoudre un programme à une dimension.

Deux difficultés demeurent.

Tout d'abord, nous devons "globaliser" la méthode, i.e. s'assurer qu'elle converge à partir de n'importe quel point de départ.

Ensuite, afin de traiter avec les contraintes d'inégalité, nous devons être capable de déterminer l'ensemble actif à la solution.